In [1]:
from __future__ import division, print_function, absolute_import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [2]:
ethnea_df = pd.read_csv('names_ethnea_genni_country_sample.csv')
#ethnea_df

In [3]:
# First thing first, use the character feature for making the dnn model
def extract_structure(word,n_char=2):
    x_struct = []
    word_len = len(word) + n_char
    n_char-=1
    counter = 0
    for i in range(word_len):
        end = i+1
        start = (i - n_char) if (i - n_char) > 0 else 0
        if word[start:end]!='_' and word[start:end]!='':
        #if word[start:end]!='_':
            x_struct.append(word[start:end])
    return x_struct

first_name_struct = ethnea_df.First.apply(lambda x: extract_structure(x.lower(),2))
last_name_struct = ethnea_df.Last.apply(lambda x: extract_structure(x.lower(),2))                                                                

In [4]:
# make struct dictionary
struct_dict = {}
for name_struct_i in first_name_struct:
    for struct_j in name_struct_i:
        if struct_j not in struct_dict:
            struct_dict[struct_j]=0
        struct_dict[struct_j]+=1
for name_struct_i in last_name_struct:
    for struct_j in name_struct_i:
        if struct_j not in struct_dict:
            struct_dict[struct_j]=0
        struct_dict[struct_j]+=1

In [5]:
struct_dict_keys = list(struct_dict.keys())
ethnic_series = ethnea_df['Ethnea'].str.lower()
pub_series = ethnea_df.PubCountry.str.lower()
ethnic_keys = list(np.unique(ethnic_series.values))
pub_keys = list(np.unique(pub_series.values))

In [6]:
# train test split
from sklearn.model_selection import train_test_split

# separate train and training set
trainIndex, testIndex, trainY, testY = train_test_split(range(len(first_name_struct)),[ethnic_keys.index(x) for x in ethnic_series],test_size = 0.2)
with open('train_test_full_index.pickle','wb') as f:
    pickle.dump((trainIndex,testIndex, trainY, testY),f)

In [7]:
def transform_structure(name_struct):
    list_structure = []
    for x in name_struct:
        try:
            list_structure.append(struct_dict_keys.index(x)+1)
        except BaseException:
            list_structure.append(0)
    #add pading 0 for structure less than num_input
    #for i in range(len(list_structure),timesteps):
    #    list_structure.append(0)
    return list_structure   
    #return [*map(lambda x:struct_dict_keys.index(x)+1, name_struct)]

#data_source = full_name_struct.apply(lambda x: transform_structure(x))

def generate_batch(first_name, last_name, i, batch_size=10000):
    len_name = len(first_name)
    #print(len_name)
    start = i*batch_size
    #print(start)
    end = start+batch_size if start+batch_size < len_name else len_name
    len_mat = end - start
    #first_name_ds_mat = np.zeros((len_mat,50),dtype=np.int32)
    #last_name_ds_mat = np.zeros((len_mat,50),dtype=np.int32)
    first_name_ds_mat = first_name[start:end]
    last_name_ds_mat = last_name[start:end]
    first_name_ds_mat = pad_sequences(first_name_ds_mat.apply(lambda x: transform_structure(x)),maxlen=50,value=0.)
    first_name_ds_mat = first_name_ds_mat.reshape(first_name_ds_mat.shape[0],1,first_name_ds_mat.shape[1])
    last_name_ds_mat = pad_sequences(last_name_ds_mat.apply(lambda x:transform_structure(x)),maxlen=50,value=0.)
    last_name_ds_mat = last_name_ds_mat.reshape(last_name_ds_mat.shape[0],1,last_name_ds_mat.shape[1])

    """
    for i in range(len_mat):
        x = first_name_struct.iloc[i]
        for y in x:
            first_name_ds_mat[i,struct_dict_keys.index(y)]+=1
        x = last_name_struct.iloc[i]
        for y in x:
            last_name_ds_mat[i,struct_dict_keys.index(y)]+=1
    """
    return first_name_ds_mat,last_name_ds_mat, range(start,end)

In [8]:
def transform_labels(x,my_keys):
    y = np.zeros(len(my_keys))
    y[my_keys.index(x)]=1
    return y


labels = np.array(list(map(lambda x: transform_labels(x,ethnic_keys),ethnic_series)))
#pub_country = np.array(list(map(lambda x: transform_labels(x,pub_keys),pub_series)))

In [11]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from tflearn.data_utils import to_categorical, pad_sequences
from keras.layers import Input
from keras.models import Model



In [30]:
# max sequence length
seq_length = 100
# multi input with single output

# first name input
first_name_input = Input(shape=(1,50),name='first_name_input')
last_name_input = Input(shape=(1,50),name='last_name_input')

first_name_l = Bidirectional(LSTM(seq_length,return_sequences=False))(first_name_input)
first_name_d = Dense(50,activation='relu')(first_name_l)
first_name_out = Dense(len(ethnic_keys),activation='softmax')(first_name_d)
last_name_l = Bidirectional(LSTM(seq_length,return_sequences=False))(last_name_input)
last_name_d = Dense(50,activation='relu')(last_name_l)
last_name_out = Dense(len(ethnic_keys),activation='softmax')(last_name_d)

# merge
merge_first_last = keras.layers.concatenate([first_name_out,last_name_out])

# final output
output_l = Dense(len(ethnic_keys),activation='softmax')(merge_first_last)

model = Model(inputs=[first_name_input,last_name_input], outputs=[first_name_out,last_name_out,output_l])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [31]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
first_name_input (InputLayer)    (None, 1, 50)         0                                            
____________________________________________________________________________________________________
last_name_input (InputLayer)     (None, 1, 50)         0                                            
____________________________________________________________________________________________________
bidirectional_7 (Bidirectional)  (None, 200)           120800      first_name_input[0][0]           
____________________________________________________________________________________________________
bidirectional_8 (Bidirectional)  (None, 200)           120800      last_name_input[0][0]            
___________________________________________________________________________________________

In [32]:
model.summary()
first_trainX = first_name_struct[trainIndex]
first_testX = first_name_struct[testIndex]
first_testX = pad_sequences(first_testX.apply(lambda x: transform_structure(x)),maxlen=50,value=0.)
first_testX = first_testX.reshape(first_testX.shape[0],1,first_testX.shape[1])
last_trainX = last_name_struct[trainIndex]
last_testX = last_name_struct[testIndex]
last_testX = pad_sequences(last_testX.apply(lambda x: transform_structure(x)),maxlen=50,value=0.)
last_testX = last_testX.reshape(last_testX.shape[0],1,last_testX.shape[1])
trainY = labels[trainIndex]
testY = labels[testIndex]

#trainX =np.array([to_categorical(x,nb_classes=len(struct_dict_keys)+1) for x in trainX])
#testX =np.array([to_categorical(x,nb_classes=len(struct_dict_keys)+1) for x in testX])

mini_batch_size = 10000
len_mini_batch = round(len(trainY)/mini_batch_size)
batch_size = 2000


for x in range(2000):
    for y in range(len_mini_batch):
        y_first_trainX, y_last_trainX, batch_range =  generate_batch(first_trainX,last_trainX,y,mini_batch_size)
        #pub_country = np.array(list(map(lambda x: transform_labels(x,pub_keys),pub_series[trainIndex].iloc[batch_range])))
        model.fit([y_first_trainX, y_last_trainX],[trainY[batch_range],trainY[batch_range],trainY[batch_range]],epochs=1,batch_size=batch_size)
    scores = model.evaluate([first_testX, last_testX],[testY, testY,testY],verbose=0)
    print('first: {}, last: {}, merge: {}'.format(scores[4],scores[5],scores[6]))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
first_name_input (InputLayer)    (None, 1, 50)         0                                            
____________________________________________________________________________________________________
last_name_input (InputLayer)     (None, 1, 50)         0                                            
____________________________________________________________________________________________________
bidirectional_7 (Bidirectional)  (None, 200)           120800      first_name_input[0][0]           
____________________________________________________________________________________________________
bidirectional_8 (Bidirectional)  (None, 200)           120800      last_name_input[0][0]            
___________________________________________________________________________________________

5653/5653 [==============================] - 0s - loss: 6.8117 - dense_17_loss: 1.9851 - dense_19_loss: 2.0073 - dense_20_loss: 2.8194 - dense_17_acc: 0.4261 - dense_19_acc: 0.4173 - dense_20_acc: 0.3879     
first: 0.4205743774373836, last: 0.40991698457222764, merge: 0.3791788198473423
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 6.7519 - dense_17_loss: 1.9526 - dense_19_loss: 1.9933 - dense_20_loss: 2.8059 - dense_17_acc: 0.4375 - dense_19_acc: 0.4224 - dense_20_acc: 0.3956      ETA: 0s - loss: 6.7130 - dense_17_loss: 1.9329 - dense_19_loss: 1.9733 - dense_20_loss: 2.8069 - dense_17_acc: 0.4510 - dense_19_acc: 0.4290 - dense_20_acc: 
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 6.7709 - dense_17_loss: 1.9708 - dense_19_loss: 2.0018 - dense_20_loss: 2.7983 - dense_17_acc: 0.4267 - dense_19_acc: 0.4149 - dense_20_acc: 0.3840     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 6.7012 - dense_17_loss: 1.9371 - dense_19_l

10000/10000 [==============================] - 0s - loss: 6.0272 - dense_17_loss: 1.7048 - dense_19_loss: 1.8326 - dense_20_loss: 2.4899 - dense_17_acc: 0.5158 - dense_19_acc: 0.4742 - dense_20_acc: 0.4437     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 6.0502 - dense_17_loss: 1.7246 - dense_19_loss: 1.8353 - dense_20_loss: 2.4903 - dense_17_acc: 0.5112 - dense_19_acc: 0.4711 - dense_20_acc: 0.4437     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 5.9748 - dense_17_loss: 1.6872 - dense_19_loss: 1.8146 - dense_20_loss: 2.4730 - dense_17_acc: 0.5165 - dense_19_acc: 0.4747 - dense_20_acc: 0.4494     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 5.9827 - dense_17_loss: 1.7010 - dense_19_loss: 1.8169 - dense_20_loss: 2.4648 - dense_17_acc: 0.5190 - dense_19_acc: 0.4863 - dense_20_acc: 0.4521     
first: 0.4906888041684572, last: 0.46185775190451395, merge: 0.4368409243752289
Epoch 1/1
10000/10000 [=========================

10000/10000 [==============================] - 0s - loss: 5.5129 - dense_17_loss: 1.5747 - dense_19_loss: 1.7235 - dense_20_loss: 2.2148 - dense_17_acc: 0.5560 - dense_19_acc: 0.4996 - dense_20_acc: 0.4890     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 5.5206 - dense_17_loss: 1.5846 - dense_19_loss: 1.7281 - dense_20_loss: 2.2080 - dense_17_acc: 0.5530 - dense_19_acc: 0.5088 - dense_20_acc: 0.4881     
first: 0.5244559120217166, last: 0.47139331388826566, merge: 0.46802782139115895
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 5.5053 - dense_17_loss: 1.5789 - dense_19_loss: 1.7263 - dense_20_loss: 2.2001 - dense_17_acc: 0.5613 - dense_19_acc: 0.5066 - dense_20_acc: 0.4823     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 5.5401 - dense_17_loss: 1.5901 - dense_19_loss: 1.7414 - dense_20_loss: 2.2085 - dense_17_acc: 0.5532 - dense_19_acc: 0.4971 - dense_20_acc: 0.4832     
Epoch 1/1
10000/10000 [========================

Epoch 1/1
10000/10000 [==============================] - 0s - loss: 5.1575 - dense_17_loss: 1.4971 - dense_19_loss: 1.6710 - dense_20_loss: 1.9894 - dense_17_acc: 0.5819 - dense_19_acc: 0.5223 - dense_20_acc: 0.5177     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 5.2032 - dense_17_loss: 1.5188 - dense_19_loss: 1.6850 - dense_20_loss: 1.9994 - dense_17_acc: 0.5768 - dense_19_acc: 0.5170 - dense_20_acc: 0.5165     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 5.1281 - dense_17_loss: 1.4845 - dense_19_loss: 1.6647 - dense_20_loss: 1.9788 - dense_17_acc: 0.5823 - dense_19_acc: 0.5190 - dense_20_acc: 0.5210     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 5.1281 - dense_17_loss: 1.4950 - dense_19_loss: 1.6600 - dense_20_loss: 1.9731 - dense_17_acc: 0.5878 - dense_19_acc: 0.5286 - dense_20_acc: 0.5236     
first: 0.5473412609645971, last: 0.4749831725910744, merge: 0.4982050706619689
Epoch 1/1
10000/10000 [================

...

Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.3005 - dense_17_loss: 0.8945 - dense_19_loss: 1.0988 - dense_20_loss: 0.3072 - dense_17_acc: 0.7546 - dense_19_acc: 0.6970 - dense_20_acc: 0.9101     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.3082 - dense_17_loss: 0.8801 - dense_19_loss: 1.1145 - dense_20_loss: 0.3135 - dense_17_acc: 0.7644 - dense_19_acc: 0.6895 - dense_20_acc: 0.9094     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.2720 - dense_17_loss: 0.8879 - dense_19_loss: 1.0769 - dense_20_loss: 0.3072 - dense_17_acc: 0.7546 - dense_19_acc: 0.7043 - dense_20_acc: 0.9105     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 2.1748 - dense_17_loss: 0.8556 - dense_19_loss: 1.0379 - dense_20_loss: 0.2813 - dense_17_acc: 0.7699 - dense_19_acc: 0.7095 - dense_20_acc: 0.9188     
first: 0.6406775858334316, last: 0.5143594346507558, merge: 0.6956472964144512
Epoch 1/1
10000/10000 [================

10000/10000 [==============================] - 0s - loss: 2.2819 - dense_17_loss: 0.8971 - dense_19_loss: 1.0782 - dense_20_loss: 0.3066 - dense_17_acc: 0.7519 - dense_19_acc: 0.7050 - dense_20_acc: 0.9120     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 2.1949 - dense_17_loss: 0.8557 - dense_19_loss: 1.0524 - dense_20_loss: 0.2868 - dense_17_acc: 0.7668 - dense_19_acc: 0.7056 - dense_20_acc: 0.9177     
first: 0.6393313888399382, last: 0.5113304913485294, merge: 0.6919452546957178
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.2891 - dense_17_loss: 0.8873 - dense_19_loss: 1.0966 - dense_20_loss: 0.3052 - dense_17_acc: 0.7551 - dense_19_acc: 0.6931 - dense_20_acc: 0.9117     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.3161 - dense_17_loss: 0.8852 - dense_19_loss: 1.1130 - dense_20_loss: 0.3179 - dense_17_acc: 0.7626 - dense_19_acc: 0.6890 - dense_20_acc: 0.9056     
Epoch 1/1
10000/10000 [==========================

Epoch 1/1
10000/10000 [==============================] - 2s - loss: 2.2918 - dense_17_loss: 0.8886 - dense_19_loss: 1.1002 - dense_20_loss: 0.3030 - dense_17_acc: 0.7545 - dense_19_acc: 0.6947 - dense_20_acc: 0.9112     
Epoch 1/1
10000/10000 [==============================] - 2s - loss: 2.3241 - dense_17_loss: 0.8913 - dense_19_loss: 1.1126 - dense_20_loss: 0.3202 - dense_17_acc: 0.7643 - dense_19_acc: 0.6884 - dense_20_acc: 0.9076     
Epoch 1/1
10000/10000 [==============================] - 2s - loss: 2.2880 - dense_17_loss: 0.8923 - dense_19_loss: 1.0881 - dense_20_loss: 0.3076 - dense_17_acc: 0.7510 - dense_19_acc: 0.7013 - dense_20_acc: 0.9099     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 2.2145 - dense_17_loss: 0.8649 - dense_19_loss: 1.0561 - dense_20_loss: 0.2936 - dense_17_acc: 0.7681 - dense_19_acc: 0.7025 - dense_20_acc: 0.9181     
first: 0.6396679380883116, last: 0.5158178147270402, merge: 0.6948620147680471
Epoch 1/1
10000/10000 [================

10000/10000 [==============================] - 1s - loss: 2.3001 - dense_17_loss: 0.9018 - dense_19_loss: 1.0848 - dense_20_loss: 0.3135 - dense_17_acc: 0.7479 - dense_19_acc: 0.7036 - dense_20_acc: 0.9076     
Epoch 1/1
5653/5653 [==============================] - 1s - loss: 2.2169 - dense_17_loss: 0.8685 - dense_19_loss: 1.0561 - dense_20_loss: 0.2924 - dense_17_acc: 0.7667 - dense_19_acc: 0.7037 - dense_20_acc: 0.9200     
first: 0.6337222346468894, last: 0.5141350684851735, merge: 0.6956472963609582
Epoch 1/1
10000/10000 [==============================] - 1s - loss: 2.3201 - dense_17_loss: 0.9088 - dense_19_loss: 1.0968 - dense_20_loss: 0.3144 - dense_17_acc: 0.7470 - dense_19_acc: 0.6953 - dense_20_acc: 0.9089     
Epoch 1/1
10000/10000 [==============================] - 1s - loss: 2.3269 - dense_17_loss: 0.8934 - dense_19_loss: 1.1120 - dense_20_loss: 0.3215 - dense_17_acc: 0.7596 - dense_19_acc: 0.6855 - dense_20_acc: 0.9026     
Epoch 1/1
10000/10000 [==========================

Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.3069 - dense_17_loss: 0.8991 - dense_19_loss: 1.0941 - dense_20_loss: 0.3138 - dense_17_acc: 0.7523 - dense_19_acc: 0.6971 - dense_20_acc: 0.9100     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.3436 - dense_17_loss: 0.8920 - dense_19_loss: 1.1202 - dense_20_loss: 0.3314 - dense_17_acc: 0.7629 - dense_19_acc: 0.6865 - dense_20_acc: 0.9044     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.2992 - dense_17_loss: 0.8981 - dense_19_loss: 1.0821 - dense_20_loss: 0.3190 - dense_17_acc: 0.7523 - dense_19_acc: 0.7021 - dense_20_acc: 0.9092     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 2.2084 - dense_17_loss: 0.8699 - dense_19_loss: 1.0497 - dense_20_loss: 0.2888 - dense_17_acc: 0.7656 - dense_19_acc: 0.7026 - dense_20_acc: 0.9174     
first: 0.6360780794389959, last: 0.5152568992595916, merge: 0.6904868745391937
Epoch 1/1
10000/10000 [================

In [ ]:
for x in range(2000):
    for y in range(len_mini_batch):
        y_first_trainX, y_last_trainX, batch_range =  generate_batch(first_trainX,last_trainX,y,mini_batch_size)
        #pub_country = np.array(list(map(lambda x: transform_labels(x,pub_keys),pub_series[trainIndex].iloc[batch_range])))
        model.fit([y_first_trainX, y_last_trainX],[trainY[batch_range],trainY[batch_range],trainY[batch_range]],epochs=1,batch_size=batch_size)
    scores = model.evaluate([first_testX, last_testX],[testY, testY,testY],verbose=0)
    print('first: {}, last: {}, merge: {}'.format(scores[4],scores[5],scores[6]))

Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.2877 - dense_17_loss: 0.9023 - dense_19_loss: 1.0844 - dense_20_loss: 0.3010 - dense_17_acc: 0.7501 - dense_19_acc: 0.7013 - dense_20_acc: 0.9114     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.3159 - dense_17_loss: 0.8943 - dense_19_loss: 1.1065 - dense_20_loss: 0.3151 - dense_17_acc: 0.7589 - dense_19_acc: 0.6932 - dense_20_acc: 0.9091     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.2730 - dense_17_loss: 0.8997 - dense_19_loss: 1.0664 - dense_20_loss: 0.3069 - dense_17_acc: 0.7501 - dense_19_acc: 0.7057 - dense_20_acc: 0.9100     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 2.1904 - dense_17_loss: 0.8631 - dense_19_loss: 1.0384 - dense_20_loss: 0.2889 - dense_17_acc: 0.7661 - dense_19_acc: 0.7097 - dense_20_acc: 0.9192     
first: 0.6389948395380718, last: 0.5181736594522804, merge: 0.6958716625265404
Epoch 1/1
10000/10000 [================

10000/10000 [==============================] - 0s - loss: 2.0790 - dense_17_loss: 0.8127 - dense_19_loss: 1.0297 - dense_20_loss: 0.2366 - dense_17_acc: 0.7747 - dense_19_acc: 0.7149 - dense_20_acc: 0.9307     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0238 - dense_17_loss: 0.8177 - dense_19_loss: 0.9892 - dense_20_loss: 0.2170 - dense_17_acc: 0.7672 - dense_19_acc: 0.7225 - dense_20_acc: 0.9346     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 1.9456 - dense_17_loss: 0.7716 - dense_19_loss: 0.9756 - dense_20_loss: 0.1984 - dense_17_acc: 0.7819 - dense_19_acc: 0.7249 - dense_20_acc: 0.9422     
first: 0.6438187121515828, last: 0.5139107022660983, merge: 0.6819609603273085
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0487 - dense_17_loss: 0.8176 - dense_19_loss: 1.0124 - dense_20_loss: 0.2188 - dense_17_acc: 0.7701 - dense_19_acc: 0.7171 - dense_20_acc: 0.9359     
Epoch 1/1
10000/10000 [==========================

5653/5653 [==============================] - 0s - loss: 1.9303 - dense_17_loss: 0.7558 - dense_19_loss: 0.9848 - dense_20_loss: 0.1897 - dense_17_acc: 0.7868 - dense_19_acc: 0.7201 - dense_20_acc: 0.9483     
first: 0.6452770921743742, last: 0.5081893650437515, merge: 0.6839802558175485
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0408 - dense_17_loss: 0.8016 - dense_19_loss: 1.0243 - dense_20_loss: 0.2149 - dense_17_acc: 0.7731 - dense_19_acc: 0.7140 - dense_20_acc: 0.9366     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0686 - dense_17_loss: 0.8086 - dense_19_loss: 1.0336 - dense_20_loss: 0.2264 - dense_17_acc: 0.7760 - dense_19_acc: 0.7105 - dense_20_acc: 0.9327     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0212 - dense_17_loss: 0.8136 - dense_19_loss: 0.9977 - dense_20_loss: 0.2100 - dense_17_acc: 0.7696 - dense_19_acc: 0.7211 - dense_20_acc: 0.9352     
Epoch 1/1
5653/5653 [============================

10000/10000 [==============================] - 0s - loss: 2.0254 - dense_17_loss: 0.7988 - dense_19_loss: 1.0200 - dense_20_loss: 0.2066 - dense_17_acc: 0.7750 - dense_19_acc: 0.7158 - dense_20_acc: 0.9379     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0417 - dense_17_loss: 0.7987 - dense_19_loss: 1.0255 - dense_20_loss: 0.2176 - dense_17_acc: 0.7816 - dense_19_acc: 0.7152 - dense_20_acc: 0.9345     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0196 - dense_17_loss: 0.8141 - dense_19_loss: 0.9943 - dense_20_loss: 0.2112 - dense_17_acc: 0.7700 - dense_19_acc: 0.7245 - dense_20_acc: 0.9359     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 1.9004 - dense_17_loss: 0.7525 - dense_19_loss: 0.9683 - dense_20_loss: 0.1796 - dense_17_acc: 0.7867 - dense_19_acc: 0.7286 - dense_20_acc: 0.9468     
first: 0.6462867399194943, last: 0.5108817590842313, merge: 0.6844289880818467
Epoch 1/1
10000/10000 [==========================

10000/10000 [==============================] - 0s - loss: 1.9828 - dense_17_loss: 0.8041 - dense_19_loss: 0.9806 - dense_20_loss: 0.1980 - dense_17_acc: 0.7724 - dense_19_acc: 0.7283 - dense_20_acc: 0.9390     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 1.8939 - dense_17_loss: 0.7481 - dense_19_loss: 0.9670 - dense_20_loss: 0.1788 - dense_17_acc: 0.7865 - dense_19_acc: 0.7302 - dense_20_acc: 0.9491     
first: 0.6455014583399565, last: 0.5113304913485294, merge: 0.6840924388334734
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 1.9980 - dense_17_loss: 0.7888 - dense_19_loss: 1.0094 - dense_20_loss: 0.1999 - dense_17_acc: 0.7775 - dense_19_acc: 0.7198 - dense_20_acc: 0.9402     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0255 - dense_17_loss: 0.7892 - dense_19_loss: 1.0230 - dense_20_loss: 0.2133 - dense_17_acc: 0.7867 - dense_19_acc: 0.7137 - dense_20_acc: 0.9373     
Epoch 1/1
10000/10000 [==========================

...

10000/10000 [==============================] - 0s - loss: 2.0331 - dense_17_loss: 0.7980 - dense_19_loss: 1.0091 - dense_20_loss: 0.2260 - dense_17_acc: 0.7713 - dense_19_acc: 0.7150 - dense_20_acc: 0.9338     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 1.9427 - dense_17_loss: 0.7427 - dense_19_loss: 0.9964 - dense_20_loss: 0.2036 - dense_17_acc: 0.7893 - dense_19_acc: 0.7235 - dense_20_acc: 0.9397     
first: 0.6452770922412406, last: 0.5076284496832889, merge: 0.6833071572673088
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0424 - dense_17_loss: 0.7847 - dense_19_loss: 1.0349 - dense_20_loss: 0.2228 - dense_17_acc: 0.7806 - dense_19_acc: 0.7107 - dense_20_acc: 0.9350     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0748 - dense_17_loss: 0.7925 - dense_19_loss: 1.0460 - dense_20_loss: 0.2364 - dense_17_acc: 0.7795 - dense_19_acc: 0.7059 - dense_20_acc: 0.9302     
Epoch 1/1
10000/10000 [==========================

Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0032 - dense_17_loss: 0.7835 - dense_19_loss: 1.0169 - dense_20_loss: 0.2027 - dense_17_acc: 0.7770 - dense_19_acc: 0.7177 - dense_20_acc: 0.9397     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0209 - dense_17_loss: 0.7791 - dense_19_loss: 1.0284 - dense_20_loss: 0.2135 - dense_17_acc: 0.7840 - dense_19_acc: 0.7122 - dense_20_acc: 0.9370     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 1.9900 - dense_17_loss: 0.7944 - dense_19_loss: 0.9895 - dense_20_loss: 0.2062 - dense_17_acc: 0.7765 - dense_19_acc: 0.7237 - dense_20_acc: 0.9400     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 1.8919 - dense_17_loss: 0.7360 - dense_19_loss: 0.9778 - dense_20_loss: 0.1781 - dense_17_acc: 0.7897 - dense_19_acc: 0.7274 - dense_20_acc: 0.9503     
first: 0.6457258245055387, last: 0.5133497869056357, merge: 0.6885797621584913
Epoch 1/1
10000/10000 [================

10000/10000 [==============================] - 0s - loss: 1.9698 - dense_17_loss: 0.7904 - dense_19_loss: 0.9791 - dense_20_loss: 0.2003 - dense_17_acc: 0.7748 - dense_19_acc: 0.7230 - dense_20_acc: 0.9398     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 1.8905 - dense_17_loss: 0.7430 - dense_19_loss: 0.9780 - dense_20_loss: 0.1695 - dense_17_acc: 0.7906 - dense_19_acc: 0.7272 - dense_20_acc: 0.9508     
first: 0.6479694862282274, last: 0.5123401391605157, merge: 0.685326452811042
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 1.9885 - dense_17_loss: 0.7821 - dense_19_loss: 1.0100 - dense_20_loss: 0.1965 - dense_17_acc: 0.7818 - dense_19_acc: 0.7191 - dense_20_acc: 0.9404     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0064 - dense_17_loss: 0.7833 - dense_19_loss: 1.0170 - dense_20_loss: 0.2061 - dense_17_acc: 0.7834 - dense_19_acc: 0.7126 - dense_20_acc: 0.9395     
Epoch 1/1
10000/10000 [===========================

Epoch 1/1
10000/10000 [==============================] - 0s - loss: 1.9935 - dense_17_loss: 0.7936 - dense_19_loss: 1.0007 - dense_20_loss: 0.1992 - dense_17_acc: 0.7757 - dense_19_acc: 0.7216 - dense_20_acc: 0.9412     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 1.9995 - dense_17_loss: 0.7863 - dense_19_loss: 1.0066 - dense_20_loss: 0.2065 - dense_17_acc: 0.7818 - dense_19_acc: 0.7197 - dense_20_acc: 0.9427     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 1.9644 - dense_17_loss: 0.8027 - dense_19_loss: 0.9685 - dense_20_loss: 0.1933 - dense_17_acc: 0.7718 - dense_19_acc: 0.7272 - dense_20_acc: 0.9418     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 1.9014 - dense_17_loss: 0.7603 - dense_19_loss: 0.9628 - dense_20_loss: 0.1783 - dense_17_acc: 0.7803 - dense_19_acc: 0.7285 - dense_20_acc: 0.9462     
first: 0.644716176827285, last: 0.5155934485614581, merge: 0.6854386358269667
Epoch 1/1
10000/10000 [=================

10000/10000 [==============================] - 0s - loss: 1.9597 - dense_17_loss: 0.7950 - dense_19_loss: 0.9711 - dense_20_loss: 0.1936 - dense_17_acc: 0.7741 - dense_19_acc: 0.7280 - dense_20_acc: 0.9426     
Epoch 1/1
5653/5653 [==============================] - 0s - loss: 1.9159 - dense_17_loss: 0.7557 - dense_19_loss: 0.9650 - dense_20_loss: 0.1952 - dense_17_acc: 0.7891 - dense_19_acc: 0.7290 - dense_20_acc: 0.9441     
first: 0.642135965856223, last: 0.5149203500179049, merge: 0.6862239174198778
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0145 - dense_17_loss: 0.8000 - dense_19_loss: 1.0026 - dense_20_loss: 0.2118 - dense_17_acc: 0.7743 - dense_19_acc: 0.7212 - dense_20_acc: 0.9374     
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 2.0289 - dense_17_loss: 0.7960 - dense_19_loss: 1.0154 - dense_20_loss: 0.2175 - dense_17_acc: 0.7799 - dense_19_acc: 0.7148 - dense_20_acc: 0.9381     
Epoch 1/1
10000/10000 [===========================